In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:5]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/invasive-species-monitoring/train_labels.csv.zip
/kaggle/input/invasive-species-monitoring/sample_submission.csv.zip
/kaggle/input/invasive-species-monitoring/test.7z
/kaggle/input/invasive-species-monitoring/train.7z
/kaggle/input/invasivespecies/test/1269.jpg
/kaggle/input/invasivespecies/test/623.jpg
/kaggle/input/invasivespecies/test/764.jpg
/kaggle/input/invasivespecies/test/1075.jpg
/kaggle/input/invasivespecies/test/771.jpg
/kaggle/input/invasivespecies/train/1269.jpg
/kaggle/input/invasivespecies/train/623.jpg
/kaggle/input/invasivespecies/train/2193.jpg
/kaggle/input/invasivespecies/train/2008.jpg
/kaggle/input/invasivespecies/train/2081.jpg


In [24]:
! unzip -q -o /kaggle/input/invasive-species-monitoring/train_labels.csv.zip
! unzip -q -o /kaggle/input/invasive-species-monitoring/sample_submission.csv.zip

In [7]:
# glob 쓰면 순서가 섞임
train = pd.read_csv('train_labels.csv')
train

,name,invasive
0,1,0
1,2,0
2,3,1
3,4,0
4,5,1
...,...,...
2290,2291,1
2291,2292,1
2292,2293,1
2293,2294,1


In [16]:
train['path'] =  '/kaggle/input/invasivespecies/train/' + train['name'].astype('str') + '.jpg'
train['invasive'] = train['invasive'].astype('str')

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_valid = train_test_split(train,test_size=0.2, random_state=2023, stratify=train['invasive'])
display(X_train,X_valid)


,name,invasive,path
182,183,1,/kaggle/input/invasivespecies/train/183.jpg
1083,1084,0,/kaggle/input/invasivespecies/train/1084.jpg
474,475,0,/kaggle/input/invasivespecies/train/475.jpg
768,769,1,/kaggle/input/invasivespecies/train/769.jpg
101,102,1,/kaggle/input/invasivespecies/train/102.jpg
...,...,...,...
2118,2119,1,/kaggle/input/invasivespecies/train/2119.jpg
472,473,0,/kaggle/input/invasivespecies/train/473.jpg
1137,1138,0,/kaggle/input/invasivespecies/train/1138.jpg
1294,1295,0,/kaggle/input/invasivespecies/train/1295.jpg


,name,invasive,path
2190,2191,1,/kaggle/input/invasivespecies/train/2191.jpg
147,148,0,/kaggle/input/invasivespecies/train/148.jpg
1548,1549,1,/kaggle/input/invasivespecies/train/1549.jpg
659,660,0,/kaggle/input/invasivespecies/train/660.jpg
673,674,0,/kaggle/input/invasivespecies/train/674.jpg
...,...,...,...
1957,1958,1,/kaggle/input/invasivespecies/train/1958.jpg
1569,1570,0,/kaggle/input/invasivespecies/train/1570.jpg
197,198,1,/kaggle/input/invasivespecies/train/198.jpg
196,197,1,/kaggle/input/invasivespecies/train/197.jpg


In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

idg = ImageDataGenerator()

train_generator = idg.flow_from_dataframe(X_train, y_col='invasive', x_col='path')
valid_generator = idg.flow_from_dataframe(X_valid, y_col='invasive', x_col='path')

Found 1836 validated image filenames belonging to 2 classes.
Found 459 validated image filenames belonging to 2 classes.


In [35]:
from tensorflow.keras import *
from tensorflow.keras.layers import *
# from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.callbacks import *
es = EarlyStopping(patience=3, restore_best_weights=True)
rl = ReduceLROnPlateau(patience=2, verbose=1) # default : 10%씩 감소

### Pretrained Model:
1. 레이어 수가 많다
2. 이미지넷 데이터셋으로 이미 학습돼있는 파라미터 보유
3. inductive bias - 보지 못한 데이터에 대해서도 귀납적으로 추론 가능하도록 하는 가정의 집합?(학습했던 객체의 label이 아니더라도 유사한 패턴 보유)

In [36]:
model = Sequential()
model.add(EfficientNetB0(include_top=False, pooling='avg'))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics='acc')


16705208/16705208 [==============================] - 1s 0us/step


## 학습오래걸리는이유
- 큰사이즈에서 256*256으로 전처리 소요시간 &rarr; validation loss 빠르게 줄이기 위해서 학습률 조정

In [37]:
# 코드분리해야 모델 초기화 안된다
model.fit(train_generator, validation_data= valid_generator,epochs=5, callbacks=[es,rl])

Epoch 1/5


2023-04-12 23:52:07.747075: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_1/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


58/58 [==============================] - 108s 1s/step - loss: 0.1664 - acc: 0.9363 - val_loss: 1.8807 - val_acc: 0.7952 - lr: 0.0010
Epoch 2/5
58/58 [==============================] - 69s 1s/step - loss: 0.0713 - acc: 0.9749 - val_loss: 0.5329 - val_acc: 0.8693 - lr: 0.0010
Epoch 3/5
58/58 [==============================] - 69s 1s/step - loss: 0.0320 - acc: 0.9864 - val_loss: 0.1111 - val_acc: 0.9782 - lr: 0.0010
Epoch 4/5
58/58 [==============================] - 68s 1s/step - loss: 0.0354 - acc: 0.9902 - val_loss: 0.0564 - val_acc: 0.9847 - lr: 0.0010
Epoch 5/5
58/58 [==============================] - 69s 1s/step - loss: 0.0299 - acc: 0.9886 - val_loss: 0.2688 - val_acc: 0.9499 - lr: 0.0010


In [38]:
test = pd.read_csv('sample_submission.csv')
test['path'] =  '/kaggle/input/invasivespecies/test/' + test['name'].astype('str') + '.jpg'


In [39]:
test_generator = idg.flow_from_dataframe(test, x_col='path', y_col=None, class_mode=None, shuffle=False)

Found 1531 validated image filenames.


In [40]:
result = model.predict(test_generator)

48/48 [==============================] - 45s 902ms/step


In [41]:
result

array([[8.4658031e-06, 9.9999154e-01],
       [9.9775106e-01, 2.2489319e-03],
       [1.7678834e-01, 8.2321173e-01],
       ...,
       [9.6141332e-01, 3.8586747e-02],
       [1.9730649e-08, 1.0000000e+00],
       [2.0090386e-12, 1.0000000e+00]], dtype=float32)

In [42]:
sub = pd.read_csv('sample_submission.csv')
sub['invasive'] = result[:,1]
sub.to_csv('sub.csv', index=False)